# Neural Network

## Навигация

1. [Предобработка](#Предобработка)
1. [One-Hot Encoding модель](#One-Hot-Encoding-модель)
1. [train/val/test разбивка и конвертация из DataFame в Dataset](#train/val/test-разбивка-и-конвертация-из-DataFame-в-Dataset)
1. [Модель нейронной сети](#Модель-нейронной-сети)
1. [SHAP values](#SHAP-values)

In [1]:
import os
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap
import tensorflow as tf

import my_ds_tools
import src

In [2]:
pd.set_option('display.max_columns', None)
tf.random.set_seed(src.constants.RANDOM_STATE)

In [3]:
TRAIN_DATA_PATH = os.path.join('..', 'data', 'processed', 'train_data.csv')
TEST_DATA_PATH = os.path.join('..', 'data', 'processed', 'test_data.csv')

TRAIN_SIZE = .7
VAL_SIZE = .2

LAYER_WIDTH = 64
BATCH_SIZE = 64
NUM_EPOCHS = 300

In [4]:
train_data = pd.read_csv(TRAIN_DATA_PATH)
test_data = pd.read_csv(TEST_DATA_PATH)

X_train = train_data.drop(columns=src.constants.TARGET)
X_test = test_data.drop(columns=src.constants.TARGET)
y_train = train_data[src.constants.TARGET]
y_test = test_data[src.constants.TARGET]

## Предобработка

[Навигация](#Навигация)

Масштабируем численные признаки:
- те, что имеют нормальное (Гаусово) распределение, стандартизируем
- другие нормализуем

In [5]:
def standanrdized(
    df: pd.DataFrame,
    feature_name: str,
    *,
    fill_nan=0,
    inplace: bool = False,
) -> pd.DataFrame:
    """Возвращает df со стандартизированным столбцом feature_name."""
    if not inplace:
        df = df.copy()

    df.loc[df[feature_name].notna(), feature_name] = (df[feature_name] - df[feature_name].mean()) / df[feature_name].std()
    df.loc[df[feature_name].isna(), feature_name] = fill_nan

    return df

In [6]:
def normalized(
    df: pd.DataFrame,
    feature_name: str,
    *,
    fill_nan=0,
    inplace: bool = False,
) -> pd.DataFrame:
    """Возвращает df с нормализованным столбцом feature_name."""
    if not inplace:
        df = df.copy()

    df.loc[df[feature_name].notna(), feature_name] = (df[feature_name] - df[feature_name].min()) / (df[feature_name].max() - df[feature_name].min())
    df.loc[df[feature_name].isna(), feature_name] = fill_nan

    return df

In [7]:
for df in [X_train, X_test]:
    df = standanrdized(df, src.constants.QUESTION_2, inplace=True)
    for i in [src.constants.QUESTION_4, src.constants.QUESTION_22, src.constants.QUESTION_24]:
        df = normalized(df, i, inplace=True)

## One-Hot Encoding модель

[Навигация](#Навигация)

One-Hot Encoding категориальных и ранговых признаков.

In [ ]:
def make_prefix(string):
    """Выделяет из названия признака только его номер."""
    prefix = ''.join(char for char in string if char.isdigit())

    if len(prefix) > 2:
        prefix = prefix[:2]

    return prefix

In [ ]:
one_hot_df = data.copy()
for feature in list(src.constants.CATEGORICAL_FEATURES) + list(src.constants.RANK_FEATURES):
    prefix = make_prefix(feature)
    one_hot_df = pd.concat(
        [one_hot_df, pd.get_dummies(one_hot_df[feature], prefix=prefix, drop_first=True)],
        axis=1
    ).drop([feature], axis=1)

Переименую также и остальные столбцы.

In [ ]:
for feature in src.constants.NUMERICAL_FEATURES + list(src.constants.BINARY_FEATURES):
    prefix = make_prefix(feature)
    one_hot_df.rename(columns={feature: prefix}, inplace=True)

## train/val/test разбивка и конвертация из DataFame в Dataset

[Навигация](#Навигация)

In [ ]:
print(
    f'Обучающая часть содержит {y_train.shape[0]} точек данных.\n'
    f'Валидационная часть содержит {y_val.shape[0]} точек данных.\n'
    f'Тестовая часть содержит {y_test.shape[0]} точек данных.'
)

In [ ]:
true_labels = y_test

In [ ]:
y_train = pd.get_dummies(y_train, prefix=src.constants.TARGET)
y_val = pd.get_dummies(y_val, prefix=src.constants.TARGET)
y_test = pd.get_dummies(y_test, prefix=src.constants.TARGET)

## Модель нейронной сети

[Навигация](#Навигация)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(LAYER_WIDTH, activation='relu', input_shape=(101,)),
    tf.keras.layers.Dropout(.5),
    tf.keras.layers.Dense(LAYER_WIDTH, activation='relu'),
    tf.keras.layers.Dropout(.5),
    tf.keras.layers.Dense(3, activation='softmax'),
])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, rankdir='TB', dpi=100)

In [ ]:
# Картинка встроена в jupyter notebook. Теперь удаляем файл, чтобы он не засорял проект.
os.remove('model.png')

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=NEURAL_NETWORK_MODEL_PATH,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
)
callbacks_list = [checkpoint]

history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    verbose=0,
    callbacks=callbacks_list,
    validation_data=(X_val, y_val),
)

In [ ]:
def plot_history(history, *, dpi=70) -> None:
    """
    Визуализирует историю обучения.

    Args:
        history: собственно история.
        dpi: количество пикселей на дюйм.

    Returns:
        Кортеж `(fig, axes)`.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 5), dpi=dpi)

    for interes, ax in zip(['accuracy', 'loss'], axes):
        ax.plot(history.history[interes], label=f'train_{interes}')
        ax.plot(history.history[f'val_{interes}'], label=f'val_{interes}')
        ax.set(xlabel='эпохи', ylabel=interes)
        ax.legend()

    return fig, axes

In [ ]:
fig, axes = plot_history(history, dpi=70)

In [ ]:
model = tf.keras.models.load_model(NEURAL_NETWORK_MODEL_PATH)

In [ ]:
pred_probabilities = model.predict(X_test)
pred_labels = []
for pred_probability in pred_probabilities:
    pred_label_index = np.array(pred_probability).argmax()
    pred_label = sorted(src.constants.LABELS)[pred_label_index]
    pred_labels.append(pred_label)
    
fig, ax = plt.subplots(figsize=(9, 6))
ConfusionMatrixDisplay.from_predictions(true_labels, pred_labels, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
plt.show()

print(classification_report(true_labels, pred_labels))

## SHAP values

[Навигация](#Навигация)

In [ ]:
background = X_train.iloc[np.random.choice(X_train.shape[0], 100, replace=False)]

In [ ]:
explainer = shap.DeepExplainer(model, background.to_numpy())
shap_values = explainer.shap_values(X.to_numpy())

In [ ]:
shap.summary_plot(shap_values, X)